In [1]:
import numpy as np
import os
from PIL import Image, features
import matplotlib.pyplot as plt
import PIL
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/introduction')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using the device",device)

%reload_ext autoreload
%autoreload 2


Using the device cuda:0


In [2]:
def load_image_as_np(image_path):
    return np.asarray(Image.open(image_path).convert('RGB'), dtype=np.uint8)

def store_image_from_np(image_path,data,format='RGBA'):
    img = Image.fromarray(data, format)
    img.save(image_path)
    return img

In [3]:
def segment_image(image,tile_size,pad_type='reflect'):
    img_height,img_width = image.shape[:2]

    # Pads the image so it can be chunked down to a grid even if the size of the image is not
    # divisible by the chunk size
    v_pad = (0,tile_size - (img_height % tile_size)) if img_height % tile_size != 0 else (0,0)
    h_pad = (0,tile_size - (img_width % tile_size)) if img_width % tile_size != 0 else (0,0)
        
    image = np.pad(image, (v_pad,h_pad,(0,0)), pad_type)

    img_height , img_width, channels = image.shape

    tiled_array =  image.reshape(img_height // tile_size,
                                 tile_size,
                                 img_width // tile_size,
                                 tile_size,
                                 channels)

    tiled_array = tiled_array.swapaxes(1,2)

    return np.concatenate(tiled_array,axis=0)


In [4]:
def rebuild_image(tile_array,image_size):
    img_height, img_width, channels = image_size
    
    tile_size = tile_array.shape[1]
    tile_rows = int(np.ceil(img_height/tile_size))
    tile_cols = int(np.ceil(img_width/tile_size))

    tile_array = tile_array.reshape(tile_rows,
                                    tile_cols,
                                    tile_size,
                                    tile_size,
                                    channels)
    #print("Rows:",chunk_rows,"Cols:",chunk_cols, "New shape:",new_shape)
    
    tile_array = np.concatenate(tile_array,axis=1)
    tile_array = np.concatenate(tile_array,axis=1)

    return tile_array[:img_height,:img_width]

In [6]:
def apply_compress_function(image, type="NONE"):
    if type=='c_net':
        return ('c_net',image)
    return ("NONE",image)


def apply_decompression_function(compressed_image,type="NONE"):
    return compressed_image
    

In [6]:
wd_path = os.path.abspath(os.getcwd())
image_path = os.path.join(wd_path,"images","tenemos.png")
image_path_out = os.path.join(wd_path,"images","tenemos_out.png")
tile_size= 8

image = load_image_as_np(image_path)

print("Original size:",image.shape)
tile_list = segment_image(image,tile_size,pad_type='reflect')
print(f"Number of {tile_size}x{tile_size} tiles:",tile_list.shape[0])

key, c_image = apply_compress_function(tile_list)
dc_image = apply_decompression_function(c_image,key)

end_image = rebuild_image(dc_image,image.shape)
print("Original and end images are equal:",(end_image==image).all())
print("With equal shape:",(end_image.shape==image.shape))

img = store_image_from_np(image_path_out,end_image,format='RGB')

#plt.imshow(end_image.astype('uint8'))
#plt.axis('off')
#plt.show()

Original size: (900, 1200, 3)
Number of 8x8 tiles: 16950
Original and end images are equal: True
With equal shape: True
